In [3]:
import numpy as np
import os
from pathlib import Path
paths_list=[]
labels_list=[]
for i in range (1, 110):
    if i != 100 and i != 92 and i != 88 :
        for j in range(3, 15):
            for r in range(0, 400):
                if os.path.isfile(f"EEG MI ROI-F/datasets/S{str(i).rjust(3, '0')}R{str(j).rjust(2, '0')}{str(r).rjust(3, '0')}.npy"):
                    paths_list.append(f"EEG MI ROI-F/datasets/S{str(i).rjust(3, '0')}R{str(j).rjust(2, '0')}{str(r).rjust(3, '0')}.npy")
labels_list = np.load('EEG MI ROI-F/labels_list.npy')
print(len(paths_list))
print(len(labels_list))
# print(paths_list)

343323
343323


In [4]:
from sklearn import model_selection
def train_test_split(series_list, labels_list):
    x_train, x_test, y_train, y_test = model_selection.train_test_split(
        series_list, labels_list, test_size=0.125, random_state=42, shuffle=True
    )
    return x_train, x_test, y_train, y_test
x_train, x_test, y_train, y_test = train_test_split(paths_list, labels_list)

In [5]:
import tensorflow as tf
import math
class EMMDSequence(tf.keras.utils.Sequence):

    def __init__(self, x_set, y_set, batch_size):
        self.x, self.y = x_set, y_set
        self.batch_size = batch_size

    def __len__(self):
        return math.ceil(len(self.x) / self.batch_size)

    def __getitem__(self, idx):
        low = idx * self.batch_size
        # Cap upper bound at array length; the last batch may be smaller
        # if the total number of items is not a multiple of batch size.
        high = min(low + self.batch_size, len(self.x))
        batch_x = self.x[low:high]
        batch_y = self.y[low:high]

        return np.array([np.load(file_name) for file_name in batch_x]), np.array(batch_y)

In [6]:
from tensorflow.keras.layers import Conv1D,MaxPooling1D,BatchNormalization,Dense,Dropout,Flatten,Input,SeparableConv1D,AveragePooling1D,SpatialDropout1D
from tensorflow.keras.models import Sequential
from tensorflow.keras.backend import clear_session
from tensorflow.keras import optimizers
num_classes = 5
sgd=optimizers.SGD( learning_rate=0.1 )
def HopefullNet():
    model = Sequential()
    clear_session()
    model.add(Conv1D(32, 20, padding='causal', activation="relu" ,input_shape = (640,2)))
    model.add(BatchNormalization())
    
    model.add(Conv1D(32, 20, padding='valid', activation="relu"))
    model.add(BatchNormalization())
    model.add(SpatialDropout1D(0.25))
    
    model.add(Conv1D(32, 6, padding='valid', activation="relu"))
    model.add(AveragePooling1D(pool_size=2, strides=1))
    
    model.add(Conv1D(32, 6, padding='valid', activation="relu"))
    model.add(SpatialDropout1D(0.25))    
    
    model.add(Flatten())
    model.add(Dense(296, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(148, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(74, activation='relu'))
    model.add(Dropout(0.25))
    model.add(Dense(num_classes, activation='softmax'))
    model.compile(sgd, loss='categorical_crossentropy', metrics=['accuracy'])
    return model

model = HopefullNet()
model.summary()

2023-09-04 15:12:36.185086: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-04 15:12:36.306057: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-04 15:12:36.306131: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-04 15:12:36.310972: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:981] could not open file to read NUMA node: /sys/bus/pci/devices/0000:01:00.0/numa_node
Your kernel may have been built without NUMA support.
2023-09-04 15:12:36.311051: I tensorflow/compile

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv1d (Conv1D)             (None, 640, 32)           1312      
                                                                 
 batch_normalization (Batch  (None, 640, 32)           128       
 Normalization)                                                  
                                                                 
 conv1d_1 (Conv1D)           (None, 621, 32)           20512     
                                                                 
 batch_normalization_1 (Bat  (None, 621, 32)           128       
 chNormalization)                                                
                                                                 
 spatial_dropout1d (Spatial  (None, 621, 32)           0         
 Dropout1D)                                                      
                                                        

In [10]:
training_generator = EMMDSequence(x_train, y_train, 512)
validation_generator = EMMDSequence(x_test, y_test, 512)
model.fit(training_generator, epochs=50, validation_data=validation_generator) 

Epoch 1/50
587/587 [==============================] - 131s 218ms/step - loss: 1.1864 - accuracy: 0.5025 - val_loss: 1.0963 - val_accuracy: 0.5370
Epoch 2/50
587/587 [==============================] - 124s 211ms/step - loss: 1.1100 - accuracy: 0.5321 - val_loss: 1.0671 - val_accuracy: 0.5551
Epoch 3/50
587/587 [==============================] - 118s 201ms/step - loss: 1.0853 - accuracy: 0.5433 - val_loss: 1.0546 - val_accuracy: 0.5556
Epoch 4/50
587/587 [==============================] - 104s 178ms/step - loss: 1.0679 - accuracy: 0.5532 - val_loss: 1.0388 - val_accuracy: 0.5637
Epoch 5/50
587/587 [==============================] - 74s 127ms/step - loss: 1.0529 - accuracy: 0.5603 - val_loss: 1.0295 - val_accuracy: 0.5714
Epoch 6/50
587/587 [==============================] - 56s 96ms/step - loss: 1.0381 - accuracy: 0.5678 - val_loss: 1.0138 - val_accuracy: 0.5773
Epoch 7/50
587/587 [==============================] - 54s 92ms/step - loss: 1.0226 - accuracy: 0.5751 - val_loss: 0.9981 - val_

In [11]:
model.save('202309041037')

INFO:tensorflow:Assets written to: 202309041037/assets


INFO:tensorflow:Assets written to: 202309041037/assets


In [7]:
import keras
model = keras.models.load_model('202309041037')
training_generator = EMMDSequence(x_train, y_train, 512)
validation_generator = EMMDSequence(x_test, y_test, 512)
model.fit(training_generator, epochs=50, validation_data=validation_generator) 

Epoch 1/50


2023-09-04 15:12:47.544318: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:432] Loaded cuDNN version 8600
2023-09-04 15:12:51.140851: I tensorflow/compiler/xla/service/service.cc:168] XLA service 0x5598cb476c10 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
2023-09-04 15:12:51.140965: I tensorflow/compiler/xla/service/service.cc:176]   StreamExecutor device (0): NVIDIA GeForce RTX 2060, Compute Capability 7.5
2023-09-04 15:12:51.277654: W tensorflow/compiler/xla/service/gpu/llvm_gpu_backend/gpu_backend_lib.cc:543] Can't find libdevice directory ${CUDA_DIR}/nvvm/libdevice. This may result in compilation or runtime failures, if the program we try to run uses routines from libdevice.
Searched for CUDA in the following directories:
  ./cuda_sdk_lib
  /usr/local/cuda-11.8
  /usr/local/cuda
  .
You can choose the search directory by setting xla_gpu_cuda_data_dir in HloModule's DebugOptions.  For most apps, setting the environment variable 

587/587 [==============================] - 166s 267ms/step - loss: 0.1082 - accuracy: 0.9638 - val_loss: 0.3350 - val_accuracy: 0.8922
Epoch 2/50
587/587 [==============================] - 137s 233ms/step - loss: 0.1053 - accuracy: 0.9641 - val_loss: 0.3370 - val_accuracy: 0.8943
Epoch 3/50
587/587 [==============================] - 144s 245ms/step - loss: 0.1024 - accuracy: 0.9654 - val_loss: 0.3232 - val_accuracy: 0.8956
Epoch 4/50
587/587 [==============================] - 105s 179ms/step - loss: 0.1003 - accuracy: 0.9664 - val_loss: 0.3318 - val_accuracy: 0.8950
Epoch 5/50
587/587 [==============================] - 76s 129ms/step - loss: 0.0995 - accuracy: 0.9666 - val_loss: 0.3269 - val_accuracy: 0.8965
Epoch 6/50
587/587 [==============================] - 65s 111ms/step - loss: 0.0965 - accuracy: 0.9676 - val_loss: 0.3255 - val_accuracy: 0.8971
Epoch 7/50
587/587 [==============================] - 57s 98ms/step - loss: 0.0943 - accuracy: 0.9686 - val_loss: 0.3182 - val_accuracy: 

In [8]:
model.save('202309041606')

INFO:tensorflow:Assets written to: 202309041606/assets


INFO:tensorflow:Assets written to: 202309041606/assets
